<a href="https://colab.research.google.com/github/aguille-vert/trump-biden-ads/blob/main/notebooks/trump_biden_download_preprocess_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Search for Trump and Biden political ads

# Initializer

In [2]:
import requests
import pandas as pd

## create facilities df

In [51]:
url='https://publicfiles.fcc.gov/api/service/facility/search/all'
parameters = {'serviceType':'tv',
              'format':'json'}
r = requests.get(url, params=parameters)
print(r)

<Response [200]>


In [48]:
facilities_df = pd.DataFrame()
for key, v in r.json()['results']['globalSearchResults'].items():
  if 'List' in key and v:
    print(key, len(v))
    df = pd.DataFrame(v)
    df['key'] = key
    facilities_df = pd.concat([facilities_df,df],
                              ignore_index=True)
facilities_df.shape

tvFacilityList 114
fmFacilityList 349
amFacilityList 139
cableSystemsList 740


(1342, 79)

In [49]:
facilities_df.dropna(subset=['id'],
                     inplace=True,
                     ignore_index=True)
facilities_df.shape

(602, 79)

In [50]:
q='trump'
trump_df = pd.DataFrame()
for row  in facilities_df.itertuples():
  try:
    entid = row.id
    print(row[0], entid)
    url = f"https://publicfiles.fcc.gov/api/manager/search/key/{q}.json?entityId={entid}"
    r = requests.get(url)
    df = pd.DataFrame(r.json()['searchResult']['files'])
    tv_id = row.id
    tv_name = row.callSign
    df['tv_id'] = tv_id
    df['tv_callSign'] = tv_name
    if df.shape[0] > 0:
      trump_df = pd.concat([trump_df,
                            df])
      print(row[0],entid, df.shape, trump_df.shape)
  except:
    pass

0 11034
1 5909
2 168560
3 59257
4 5914
5 17399
6 48659
7 6864
8 17433
9 83306
10 53324
11 55370
12 132606
13 86208
14 60728
15 22201
16 17037
17 33770
18 60740
19 41975
20 55379
21 55375
22 60736
23 61064
24 53329
25 49326
26 67910
27 41983
28 49324
29 61067
30 34412
31 65370
32 53321
33 51429
34 61335
35 29015
36 53320
37 56028
38 66258
39 62427
40 62430
41 13792
42 53315
43 55364
44 7675
45 16930
46 162016
47 73701
48 35200
49 32958
49 32958 (9, 12) (9, 12)
50 55362
51 69692
52 50590
53 8260
54 71072
55 8284
56 86205
57 41964
58 48660
59 1270
59 1270 (12, 12) (21, 12)
60 68834
61 61063
62 49134
63 56971
64 35567
64 35567 (5, 12) (26, 12)
65 35585
65 35585 (4, 12) (30, 12)
66 32176
67 29121
68 48658
69 71558
70 53313
71 60534
72 35645
73 61350
74 28280
75 47996
76 61066
77 28521
78 22570
78 22570 (6, 12) (36, 12)
79 23422
80 51517
81 42359
82 169028
83 61072
84 63927
84 63927 (45, 12) (81, 12)
85 35841
86 61961
86 61961 (88, 12) (169, 12)
87 55372
88 78910
89 776246
90 53318
91 49330
